In [100]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Flatten, Dense, GlobalMaxPooling2D
from keras.models import Model
from keras.utils import to_categorical

### Data pre-processing

In [101]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()

In [102]:
print(X_train.shape)
number_of_sample = 20
first_twenty = []
for i in range(number_of_sample):
  first_twenty.append(i)

mask = np.isin(y_train, first_twenty)
y_train_all = y_train[mask]
X_train_all = X_train[mask.squeeze()]


mask = np.isin(y_test, first_twenty)
X_test = X_test[mask.squeeze()]
y_test = y_test[mask]

print(X_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 32, 32, 3)
(2000,)


In [103]:
X_train, y_train = X_train_all[ : int(len(X_train_all) * 0.9)], y_train_all[ : int(len(X_train_all) * 0.9)]
X_val, y_val = X_train_all[int(len(X_train_all) * 0.9) : ], y_train_all[int(len(X_train_all) * 0.9) : ]

In [104]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

(9000, 32, 32, 3) (9000,)
(2000, 32, 32, 3) (2000,)
(1000, 32, 32, 3) (1000,)


### Model number1 : RestNet50

In [105]:
from keras.applications import ResNet50

In [106]:
# def preprocess(images, label):
#   images = tf.image.resize(images, [224, 224])
#   images = images / 255.0
#   return images, label

In [107]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# train_dataset = train_dataset.map(preprocess).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
# val_dataset = val_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
# test_dataset = test_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [108]:
resnet = ResNet50(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
resnet.trainable = False
x = resnet.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(resnet.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_4344']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.3591 - loss: 2.5457 - val_accuracy: 0.4690 - val_loss: 1.7299
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6087 - loss: 1.2359 - val_accuracy: 0.4850 - val_loss: 1.6932
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7106 - loss: 0.8989 - val_accuracy: 0.4990 - val_loss: 1.7170
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7986 - loss: 0.6520 - val_accuracy: 0.4910 - val_loss: 1.8568
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8464 - loss: 0.4973 - val_accuracy: 0.5070 - val_loss: 1.9566


In [109]:
import os

In [110]:

model.save("resnet50_model.h5")

size_in_bytes = os.path.getsize("resnet50_model.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"ResNet50 model size: {size_in_mb:.2f} MB")

ResNet50 model size: 96.60 MB


In [111]:
final_epoch = -1  # last epoch

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5678 - loss: 1.7036
Training Loss: 0.5218
Training Accuracy: 0.8346
Validation Loss: 1.9566
Validation Accuracy: 0.5070
Test Loss: 1.7244
Test Accuracy: 0.5550


###2. VGG16

In [112]:
from keras.applications import VGG16

In [113]:
vgg16 = VGG16(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
vgg16.trainable = False
x = vgg16.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(vgg16.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_4522']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.2929 - loss: 10.6267 - val_accuracy: 0.3930 - val_loss: 3.1374
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.5247 - loss: 1.9105 - val_accuracy: 0.4270 - val_loss: 2.6420
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6589 - loss: 1.1452 - val_accuracy: 0.4690 - val_loss: 2.6124
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7654 - loss: 0.7680 - val_accuracy: 0.4580 - val_loss: 2.7490
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8201 - loss: 0.5771 - val_accuracy: 0.4730 - val_loss: 2.6973


In [114]:
final_epoch = -1  # last epoch

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4966 - loss: 2.3978
Training Loss: 0.5843
Training Accuracy: 0.8157
Validation Loss: 2.6973
Validation Accuracy: 0.4730
Test Loss: 2.3798
Test Accuracy: 0.4915


In [115]:

model.save("vgg16.h5")

size_in_bytes = os.path.getsize("vgg16.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"VGG model size: {size_in_mb:.2f} MB")

VGG model size: 57.78 MB


###3. VGG19

In [116]:
from keras.applications import VGG19

In [117]:
vgg19 = VGG19(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
vgg19.trainable = False
x = vgg19.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(vgg19.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_4544']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.2893 - loss: 10.9077 - val_accuracy: 0.3820 - val_loss: 3.0657
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4948 - loss: 2.0053 - val_accuracy: 0.4390 - val_loss: 2.5622
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6182 - loss: 1.3006 - val_accuracy: 0.4490 - val_loss: 2.5139
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7140 - loss: 0.9220 - val_accuracy: 0.4600 - val_loss: 2.5771
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7843 - loss: 0.6945 - val_accuracy: 0.4940 - val_loss: 2.5746


In [118]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4593 - loss: 2.3763
Training Loss: 0.7511
Training Accuracy: 0.7652
Validation Loss: 2.5746
Validation Accuracy: 0.4940
Test Loss: 2.3520
Test Accuracy: 0.4735


In [119]:

model.save("vgg19.h5")

size_in_bytes = os.path.getsize("vgg19.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"VGG model size: {size_in_mb:.2f} MB")

VGG model size: 78.05 MB


### 4. DenseNet

In [120]:
from keras.applications import DenseNet121

In [121]:
densenet = DenseNet121(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
densenet.trainable = False
x = densenet.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(densenet.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_4569']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 56s 122ms/step - accuracy: 0.2053 - loss: 5.6879 - val_accuracy: 0.3470 - val_loss: 2.1600
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 44s 14ms/step - accuracy: 0.3934 - loss: 2.0198 - val_accuracy: 0.4100 - val_loss: 1.9658
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4549 - loss: 1.7655 - val_accuracy: 0.3950 - val_loss: 1.9560
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5022 - loss: 1.6047 - val_accuracy: 0.4040 - val_loss: 1.9334
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5222 - loss: 1.5256 - val_accuracy: 0.4470 - val_loss: 1.8317


In [122]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.4638 - loss: 1.7631
Training Loss: 1.5300
Training Accuracy: 0.5201
Validation Loss: 1.8317
Validation Accuracy: 0.4470
Test Loss: 1.7528
Test Accuracy: 0.4570


In [123]:

model.save("densnet121.h5")

size_in_bytes = os.path.getsize("densnet121.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"VGG model size: {size_in_mb:.2f} MB")

VGG model size: 31.17 MB


### 5. MobileNet

In [124]:
from keras.applications import MobileNet

In [125]:
mobilenet = MobileNet(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
mobilenet.trainable = False
x = mobilenet.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(mobilenet.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

<ipython-input-125-b63ec3278128>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = MobileNet(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_4999']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.0723 - loss: 2.9724 - val_accuracy: 0.1260 - val_loss: 2.8920
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1139 - loss: 2.8850 - val_accuracy: 0.1150 - val_loss: 2.8619
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1235 - loss: 2.8562 - val_accuracy: 0.1260 - val_loss: 2.8535
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1239 - loss: 2.8304 - val_accuracy: 0.1240 - val_loss: 2.8477
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1263 - loss: 2.8319 - val_accuracy: 0.1330 - val_loss: 2.8321


In [126]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1140 - loss: 2.8668
Training Loss: 2.8276
Training Accuracy: 0.1293
Validation Loss: 2.8321
Validation Accuracy: 0.1330
Test Loss: 2.8546


In [127]:
model.save("mobilenet.h5")

size_in_bytes = os.path.getsize("mobilenet.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"mobilenet model size: {size_in_mb:.2f} MB")

mobilenet model size: 15.66 MB


### 6. MobileNetV2

In [128]:
from keras.applications import MobileNetV2

In [129]:
mobilenetV2 = MobileNetV2(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
mobilenetV2.trainable = False
x = mobilenetV2.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(mobilenetV2.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

<ipython-input-129-c00abd5189d2>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenetV2 = MobileNetV2(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_5088']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.0821 - loss: 2.9619 - val_accuracy: 0.1160 - val_loss: 2.8752
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1259 - loss: 2.8563 - val_accuracy: 0.1310 - val_loss: 2.8521
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1343 - loss: 2.8167 - val_accuracy: 0.1200 - val_loss: 2.8380
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1458 - loss: 2.7901 - val_accuracy: 0.1290 - val_loss: 2.8390
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.1431 - loss: 2.7779 - val_accuracy: 0.1330 - val_loss: 2.8419


In [130]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.1288 - loss: 2.8225
Training Loss: 2.7683
Training Accuracy: 0.1467
Validation Loss: 2.8419
Validation Accuracy: 0.1330
Test Loss: 2.8329


In [131]:
model.save("mobilenetv2.h5")

size_in_bytes = os.path.getsize("mobilenetv2.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"mobilenetv2 model size: {size_in_mb:.2f} MB")

mobilenetv2 model size: 12.94 MB


###7. NASNetMobile

In [132]:
from keras.applications import NASNetMobile

In [133]:
nasnetmobile = NASNetMobile(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
nasnetmobile.trainable = False
x = nasnetmobile.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(nasnetmobile.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_5245']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 65s 117ms/step - accuracy: 0.1001 - loss: 85.1964 - val_accuracy: 0.0640 - val_loss: 3.1525
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 45s 20ms/step - accuracy: 0.0523 - loss: 3.0397 - val_accuracy: 0.0510 - val_loss: 2.9975
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0459 - loss: 2.9888 - val_accuracy: 0.0450 - val_loss: 2.9996
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0565 - loss: 2.9875 - val_accuracy: 0.0440 - val_loss: 2.9956
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.0554 - loss: 2.9861 - val_accuracy: 0.0470 - val_loss: 2.9927


In [134]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.0546 - loss: 3.0064
Training Loss: 2.9892
Training Accuracy: 0.0540
Validation Loss: 2.9927
Validation Accuracy: 0.0470
Test Loss: 3.0084


In [135]:
model.save("nasnetmobile.h5")

size_in_bytes = os.path.getsize("nasnetmobile.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"nasnetmobile model size: {size_in_mb:.2f} MB")

nasnetmobile model size: 22.86 MB


###8. ResNet101

In [136]:
from keras.applications import ResNet101

In [137]:
resnet101 = ResNet101(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
resnet101.trainable = False
x = resnet101.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(resnet101.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6017']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)


282/282 ━━━━━━━━━━━━━━━━━━━━ 36s 65ms/step - accuracy: 0.3567 - loss: 2.5985 - val_accuracy: 0.4780 - val_loss: 1.6958
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5989 - loss: 1.2787 - val_accuracy: 0.5070 - val_loss: 1.7614
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.7031 - loss: 0.9407 - val_accuracy: 0.5160 - val_loss: 1.7257
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.7690 - loss: 0.7112 - val_accuracy: 0.5500 - val_loss: 1.7620
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8190 - loss: 0.5457 - val_accuracy: 0.5320 - val_loss: 1.9559


In [138]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.5648 - loss: 1.7841
Training Loss: 0.5501
Training Accuracy: 0.8201
Validation Loss: 1.9559
Validation Accuracy: 0.5320
Test Loss: 1.8205


In [139]:
model.save("resnet101.h5")

size_in_bytes = os.path.getsize("resnet101.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"resnet101 model size: {size_in_mb:.2f} MB")

resnet101 model size: 169.88 MB


###9. ConvNextSamll

In [140]:
from keras.applications import ConvNeXtSmall

In [141]:
convnextsmall = ConvNeXtSmall(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
convnextsmall.trainable = False
x = convnextsmall.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(convnextsmall.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 34s 53ms/step - accuracy: 0.5645 - loss: 1.4917 - val_accuracy: 0.6680 - val_loss: 1.0118
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8272 - loss: 0.5570 - val_accuracy: 0.6920 - val_loss: 1.0113
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8925 - loss: 0.3457 - val_accuracy: 0.7020 - val_loss: 1.0092
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9414 - loss: 0.2201 - val_accuracy: 0.7020 - val_loss: 1.0352
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9661 - loss: 0.1363 - val_accuracy: 0.7010 - val_loss: 1.0987


In [142]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.7308 - loss: 0.9438
Training Loss: 0.1424
Training Accuracy: 0.9637
Validation Loss: 1.0987
Validation Accuracy: 0.7010
Test Loss: 0.9625


In [143]:
model.save("convnextsmall.h5")

size_in_bytes = os.path.getsize("convnextsmall.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"convnextsmall model size: {size_in_mb:.2f} MB")

convnextsmall model size: 191.89 MB


### 10. ConvNeXtBase

In [144]:
from keras.applications import ConvNeXtBase

In [145]:
convnextbase = ConvNeXtBase(input_tensor = Input(shape=(32, 32, 3)), weights='imagenet', include_top=False)
convnextbase.trainable = False
x = convnextbase.output
x = GlobalMaxPooling2D()(x)
x=Dense(256,activation='relu')(x)
outputs = Dense(20, activation = 'softmax', name = 'output-layer')(x)
model = Model(convnextbase.input, outputs, name = 'ResNet')
model.compile(optimizer=keras.optimizers.Adam(0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 37s 56ms/step - accuracy: 0.6054 - loss: 1.3209 - val_accuracy: 0.7400 - val_loss: 0.8291
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8812 - loss: 0.3712 - val_accuracy: 0.7380 - val_loss: 0.8467
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9576 - loss: 0.1699 - val_accuracy: 0.7410 - val_loss: 0.8896
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9826 - loss: 0.0906 - val_accuracy: 0.7350 - val_loss: 0.9394
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9888 - loss: 0.0511 - val_accuracy: 0.7480 - val_loss: 0.9742


In [146]:
final_epoch = -1

train_loss = history.history['loss'][final_epoch]
train_accuracy = history.history['accuracy'][final_epoch]
val_loss = history.history['val_loss'][final_epoch]
val_accuracy = history.history['val_accuracy'][final_epoch]
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.7751 - loss: 0.8358
Training Loss: 0.0524
Training Accuracy: 0.9873
Validation Loss: 0.9742
Validation Accuracy: 0.7480
Test Loss: 0.8485


In [147]:
model.save("convnextbase.h5")

size_in_bytes = os.path.getsize("convnextbase.h5")
size_in_mb = size_in_bytes / (1024 * 1024)
print(f"convnextbase model size: {size_in_mb:.2f} MB")

convnextbase model size: 337.96 MB
